In [3]:
import requests
import re
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv

def download_page(pageUrl):
    try:
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
        page = requests.get(pageUrl, 
                             headers={'User-Agent':user_agent}) 
        html = page.content
        contents = html.decode('utf-8')
        soup = BeautifulSoup(contents, 'html.parser')
        
        title = soup.find('h1', {'class': 'article-title'})
        rubric = soup.find('li', {'class': 'item-meta meta-flow'})
        obj = soup.find('li', {'class': 'item-meta meta-posted'})
        views = soup.find('li', {'class': 'item-meta meta-views-counter'})
        comments = soup.find('li', {'class': 'item-meta meta-comments-counter'})
        content = soup.find('div', {'class': 'article-wrap'})
        
        return title, rubric, obj, views, comments, content
    except requests.HTTPError:
        return 'HTTPError at %s'%pageUrl
    except UnicodeEncodeError:
        pass
    except ConnectionError:
        time.sleep(30)
        pass

titles = []
rubrics = []
objects = []
viewsCount = []
commentsCount = []
contents = []
ids = []
commonUrl = 'http://www.the-village.ru/village/city/situation/'
for i in tqdm(range(280000, 297134)):
    pageUrl = commonUrl + str(i)
    if download_page(pageUrl) == (None, None, None, None, None, None):
        continue
    else:
        titles.append('%s'%download_page(pageUrl)[0])
        rubrics.append('%s'%download_page(pageUrl)[1])
        objects.append('%s'%download_page(pageUrl)[2])
        viewsCount.append('%s'%download_page(pageUrl)[3])
        commentsCount.append('%s'%download_page(pageUrl)[4])
        contents.append('%s'%download_page(pageUrl)[5])
        ids.append('%d'%i)
cleanTitles = []
cleanRubrics = []
cleanObjects = []
cleanestObjects =[]
cleanViewsCount = []
cleanCommentsCount = []
cleanContents = []
cleanestContents = []
for item in titles:
    data = re.sub(r'\<[^>]*\>', '', item)
    cleanTitles.append(data)
for item in rubrics:
    data = re.sub(r'\<[^>]*\>', '', item)
    cleanRubrics.append(data)
for item in objects:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanObjects.append(data)
for item in cleanObjects:
    data = re.sub(u'\\xa0', ' ', item)
    cleanestObjects.append(data)
for item in viewsCount:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanViewsCount.append(data)
for item in commentsCount:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanCommentsCount.append(data)
for item in contents:
    data = re.sub(r'\<[^>]*\>', ' ', item)
    cleanContents.append(data)
for item in cleanContents:
    data = re.sub(u'\\xa0', ' ', item)
    cleanestContents.append(data)
authors = []
dates = []
splObjects = []
for el in cleanestObjects:
    splObj = el.split(',')
    splObjects.append(splObj)
for el in splObjects:
    try:
        authors.append(el[0])
    except IndexError:
        authors.append('???')
    try:
        dates.append(el[1])
    except IndexError:
        dates.append('???')
raw_data = {
    'ID': ids,
    'title': cleanTitles,
    'rubric': cleanRubrics,
    'author': authors,
    'date': dates,
    'content': cleanestContents,
    'views': cleanViewsCount,
    'comments': cleanCommentsCount,
    }
df = pd.DataFrame(raw_data, columns = ['ID', 'title', 'rubric', 'author', 'date', 'content', 'views', 'comments'])
df.to_csv('village_city_situation1.csv', encoding='utf-8')

A Jupyter Widget

In [43]:
df = pd.read_csv('village_city_situation1.csv', engine='python', encoding='utf-8', index_col=0, na_values='')
df

,ID,title,rubric,author,date,content,views,comments
0,280000,Бывший вице-губернатор Приморья получил условн...,Новости,Dmitry Rodinko,23 августа 2017 в 09:21,\n \n\n Фрунзенский суд Владивостока при...,134.0,0.0
1,280004,В аэропортах Москвы вновь отменили и задержали...,Новости,Роман Чернянский,23 августа 2017 в 09:40,"\n \n\n Утром в среду, 23 августа, в аэр...",1329.0,0.0
2,280006,Cколько екатеринбуржцев готовы нарушить закон ...,Цифра дня,Светлана Дюмаева,23 августа 2017 в 09:39,\n \n\n Сотрудники портала «Зарплата.ру»...,164.0,0.0
3,280008,На Почаине откроется интерактивный музей в тем...,Город,Марк Григорьев,23 августа 2017 в 09:52,\n \n\n Музей «Смешанные чувства» открое...,265.0,0.0
4,280010,Каким екатеринбуржцы видят будущее России,Видео дня,Светлана Дюмаева,23 августа 2017 в 10:01,\n \n\n Портал Znak.com поговорил с жите...,109.0,0.0
5,280012,На Таганско-Краснопресненской линии метро увел...,Новости,Роман Чернянский,23 августа 2017 в 10:04,"\n \n\n Утром в среду, 23 августа, на Та...",1599.0,0.0
6,280014,Фестиваль зрелищных фильмов IMAX пройдет в кин...,Новости,Dmitry Rodinko,23 августа 2017 в 10:15,\n \n\n C 24 по 30 августа в кинотеатре ...,319.0,0.0
7,280016,«Бертгольд Центр» запускает цикл встреч с деят...,Новости,Сергей Феофанов,23 августа 2017 в 10:41,\n \n\n «Бертгольд Центр» запускает прое...,529.0,0.0
8,280018,Сколько денег россияне хранят в офшорах,Цифра дня,Роман Чернянский,23 августа 2017 в 10:47,\n \n\n Американская некоммерческая орга...,1305.0,0.0
9,280020,Гид The VillageМесто для игры: какие локации Н...,None,None,???,\n \n \n \n\n В день выпуска фин...,1495.0,1.0


In [44]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import pymorphy2

def tokenizer(text):
    if not(text): return ''
    return re.findall('[a-zа-я]+', text.lower())

        
def lemmatizer(tokens):
    morph = pymorphy2.MorphAnalyzer()
    sws = set(stopwords.words('russian'))
    tokens_lemma = []
    for token in tokens:
        token_lemma = morph.parse(token)[0].normal_form
        if not(token in sws):
            tokens_lemma.append(token_lemma)
    return tokens_lemma

In [46]:
contents = df['content'].values
contents_tokens = [tokenizer(content) for content in tqdm(contents)]

contents_lemma = [' '.join(lemmatizer(tokens)) for tokens in tqdm(contents_tokens)]
contents_lemma

A Jupyter Widget

A Jupyter Widget

['фрунзенский суд владивосток приговорить бывший вица губернатор приморье сергей сидоренко условный наказание срок половина год сообщать интерфакс чиновник осудить махинация распределение материальный помощь аграрий потерять урожай год время тайфун лайонрок суд признать сидоренко виновный превышение должностной полномочие помимо условный срок сидоренко также год лишить право занимать должность государственный служба ранее такой срок вид наказание просить суд представитель гособвинение экс чиновник рассчитывать штраф оглашение приговор сидоренко заявить журналист считать наказание слишком суровый возможно обжаловать ущерб пострадавший соответственно почему такой решение понимать сказать данные следствие сентябрь год сидоренко курировать вопрос сельский хозяйство объявить территория артемовский городской округа режим чс следствие полагать сделать это несколько коммерческий организация один который руководить родственник получить федеральный бюджет деньга предназначить аграрий понести уще

In [52]:
with open ('moscow_districts_e.txt', encoding='utf-8') as fl:
    districts = fl.read()
districtsList = districts.split('\n')
print(districtsList)

['\ufeffАкадемическое', 'Алексеевское', 'Алтуфьевское', 'Арбат', 'Аэропорт', 'Бабушкинское', 'Басманное', 'Беговое', 'Бескудниковское', 'Бибирево', 'Бирюлево Восточное', 'Бирюлево Западное', 'Богородское', 'Братеево', 'Бутырское', 'Вешняки', 'Внуково', 'Войковское', 'Восточное Дегунино', 'Восточное Измайлово', 'Восточное', 'Выхино-Жулебино', 'Гагаринское', 'Головинское', 'Гольяново', 'Даниловское', 'Дмитровское', 'Донское', 'Дорогомилово', 'Замоскворечье', 'Западное Дегунино', 'Зюзино', 'Зябликово', 'Ивановское', 'Измайлово', 'Капотня', 'Коньково', 'Коптево', 'Косино-Ухтомское', 'Котловка', 'Красносельское', 'Крылатское', 'Крюково', 'Кузьминки', 'Кунцево', 'Куркино', 'Левобережное', 'Лефортово', 'Лианозово', 'Ломоносовское', 'Лосиноостровское', 'Люблино', 'Марфино', 'Марьина Роща', 'Марьино', 'Матушкино', 'Метрогородок', 'Мещанское', 'Митино', 'Можайское', 'Молжаниновское', 'Москворечье-Сабурово', 'Нагатино-Садовники', 'Нагатинский Затон', 'Нагорное', 'Некрасовка', 'Нижегородское', 'Но

In [53]:
def tokenizer(text):
    return re.findall('[a-zа-я]+', text.lower())

        
def lemmatizer(tokens):
    morph = pymorphy2.MorphAnalyzer()
    sws = set(stopwords.words('russian'))
    tokens_lemma = []
    for token in tokens:
        token_lemma = morph.parse(token)[0].normal_form
        if not(token in sws):
            tokens_lemma.append(token_lemma)
    return tokens_lemma

In [54]:
districts_tokens = [tokenizer(district) for district in districtsList]

districts_lemma = [' '.join(lemmatizer(tokens)) for tokens in districts_tokens]
districts_lemma

['академический',
 'алексеевский',
 'алтуфьевский',
 'арбат',
 'аэропорт',
 'бабушкинский',
 'басманный',
 'беговой',
 'бескудниковский',
 'бибирево',
 'бирюлево восточный',
 'бирюлево западный',
 'богородский',
 'братеево',
 'бутырский',
 'вешняк',
 'внуково',
 'войковский',
 'восточный дегунино',
 'восточный измайлово',
 'восточный',
 'выхино жулебино',
 'гагаринский',
 'головинский',
 'гольяново',
 'даниловский',
 'дмитровский',
 'донский',
 'дорогомилово',
 'замоскворечье',
 'западный дегунино',
 'зюзино',
 'зябликово',
 'ивановский',
 'измайлово',
 'капотня',
 'коньковый',
 'коптево',
 'косино ухтомский',
 'котловка',
 'красносельский',
 'крылатский',
 'крюково',
 'кузьминка',
 'кунцево',
 'куркино',
 'левобережный',
 'лефортово',
 'лианозово',
 'ломоносовский',
 'лосиноостровский',
 'люблино',
 'марфино',
 'марьин роща',
 'марьино',
 'матушкин',
 'метрогородок',
 'мещанский',
 'митино',
 'можайский',
 'молжаниновский',
 'москворечье сабуровый',
 'нагатино садовник',
 'нагатинский

In [56]:
with open ('mosdistricts_lemma.txt', 'w') as fl:
    districts = '\n'.join(districts_lemma)
    fl.write(districts)
    fl.close

In [58]:
df['lemmas']= contents_lemma
df

        
    

,ID,title,rubric,author,date,content,views,comments,lemmas
0,280000,Бывший вице-губернатор Приморья получил условн...,Новости,Dmitry Rodinko,23 августа 2017 в 09:21,\n \n\n Фрунзенский суд Владивостока при...,134.0,0.0,фрунзенский суд владивосток приговорить бывший...
1,280004,В аэропортах Москвы вновь отменили и задержали...,Новости,Роман Чернянский,23 августа 2017 в 09:40,"\n \n\n Утром в среду, 23 августа, в аэр...",1329.0,0.0,утром среда август аэропорт москва отменить за...
2,280006,Cколько екатеринбуржцев готовы нарушить закон ...,Цифра дня,Светлана Дюмаева,23 августа 2017 в 09:39,\n \n\n Сотрудники портала «Зарплата.ру»...,164.0,0.0,сотрудник портал зарплата ру поговорить житель...
3,280008,На Почаине откроется интерактивный музей в тем...,Город,Марк Григорьев,23 августа 2017 в 09:52,\n \n\n Музей «Смешанные чувства» открое...,265.0,0.0,музей смешанный чувство открыться почаинский к...
4,280010,Каким екатеринбуржцы видят будущее России,Видео дня,Светлана Дюмаева,23 августа 2017 в 10:01,\n \n\n Портал Znak.com поговорил с жите...,109.0,0.0,портал znak com поговорить житель екатеринбург...
5,280012,На Таганско-Краснопресненской линии метро увел...,Новости,Роман Чернянский,23 августа 2017 в 10:04,"\n \n\n Утром в среду, 23 августа, на Та...",1599.0,0.0,утром среда август таганско краснопресненский ...
6,280014,Фестиваль зрелищных фильмов IMAX пройдет в кин...,Новости,Dmitry Rodinko,23 августа 2017 в 10:15,\n \n\n C 24 по 30 августа в кинотеатре ...,319.0,0.0,c август кинотеатр океан набережная пройти фес...
7,280016,«Бертгольд Центр» запускает цикл встреч с деят...,Новости,Сергей Феофанов,23 августа 2017 в 10:41,\n \n\n «Бертгольд Центр» запускает прое...,529.0,0.0,бертгольд центр запускать проект ресурс цикл в...
8,280018,Сколько денег россияне хранят в офшорах,Цифра дня,Роман Чернянский,23 августа 2017 в 10:47,\n \n\n Американская некоммерческая орга...,1305.0,0.0,американский некоммерческий организация национ...
9,280020,Гид The VillageМесто для игры: какие локации Н...,None,None,???,\n \n \n \n\n В день выпуска фин...,1495.0,1.0,день выпуск финальный эпизод семь сезон культо...


In [136]:
districtsDict = dict.fromkeys(districts_lemma, )
for i in range(0,100):
    words = df.lemmas.iloc[i]
    wordSet = set(words.split(' '))
    for key in districtsDict:
        for word in wordSet:
            if word == key:
                districtsDict.update({key: list(districtsDict[key]).append(df.ID.iloc[i])})
print(districtsDict)

TypeError: 'NoneType' object is not iterable